# **User 관심도 점수를 카드 혜택에 맞춰 카드별 매칭 점수 도출**
# 1. 필요한 라이브러리 및 데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
card = pd.read_csv('/content/drive/MyDrive/CARD-LANGCHAIN/data/card_data.csv')
interests = pd.read_csv('/content/drive/MyDrive/CARD-LANGCHAIN/data/관심도모델링1차데이터.csv')

# 2. 카드 혜택 대분류 카테고리 뽑기

In [3]:
card.head()

,Card Name,Annual_fee,Base Record,Category_List
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,['항공마일리지']
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"['쇼핑', '레저', '카페/베이커리', '편의점', '대형마트', '의료', '..."
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"['쇼핑', '주유', '카페/베이커리', '편의점', '대중교통', '간편결제',..."
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"['대중교통', '간편결제', '편의점', '통신', '카페/베이커리', '대형마트..."
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"['포인트/캐시백', '카페/베이커리', '쇼핑', '통신', '대중교통', '영화..."


In [4]:
import ast
card['Category_List'] = card['Category_List'].apply(ast.literal_eval)

In [5]:
category_dict = {
'항공' : '항공마일리지',
'쇼핑/간편결제' : ['쇼핑', '간편결제' , '포인트/캐시백', '편의점', '대형마트'],
'카페/베이커리' :['카페', '베이커리', '외식'],
'의료' : '의료'
,'반려동물': '반려동물'
,'뷰티': '뷰티'
,'교통/주유' : ['대중교통', '주유', '하이패스'],
'교육/육아': ['교육', '육아'],
'문화': ['문화', '레저', '영화'],
'납부': ['통신', '관리비', '납부 혜택']}

# 매핑 함수
def map_category_list_to_key(category_list):
  keys = set()
  for category in category_list:
    for key, values in category_dict.items():
        if category in values:
            keys.add(key)
  return list(keys)

# 대분류 카테고리 컬럼 추가
card['category'] = card['Category_List'].apply(map_category_list_to_key)

card.head()

,Card Name,Annual_fee,Base Record,Category_List,category
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,[항공마일리지],[항공]
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"[쇼핑, 레저, 카페/베이커리, 편의점, 대형마트, 의료, 뷰티, 대중교통, 외식]","[뷰티, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화]"
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"[쇼핑, 주유, 카페/베이커리, 편의점, 대중교통, 간편결제, 대형마트, 외식, 통...","[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 항공..."
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"[대중교통, 간편결제, 편의점, 통신, 카페/베이커리, 대형마트, 주유, 외식, 쇼...","[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화..."
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"[포인트/캐시백, 카페/베이커리, 쇼핑, 통신, 대중교통, 영화, 편의점]","[쇼핑/간편결제, 납부, 문화, 교통/주유]"


# 3. 혜택 별 카드 매칭

In [6]:
card['category']

,category
0,[항공]
1,"[뷰티, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화]"
2,"[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 항공..."
3,"[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화..."
4,"[쇼핑/간편결제, 납부, 문화, 교통/주유]"
...,...
161,"[뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아]"
162,[항공]
163,"[쇼핑/간편결제, 카페/베이커리, 문화]"
164,"[쇼핑/간편결제, 교통/주유]"


In [7]:
interests.head()

,user_id,category,암묵적관심도,명시적관심도
0,1,교육/육아,1,0
1,1,문화,0,1
2,1,납부,0,1
3,1,항공,0,1
4,1,반려동물,1,0


user_id별로 관심 카테고리가 있는 카드 혜택을 매칭시켜
암묵적 관심도는 *0.7, 명시적관심도 * 0.9 를 곱해
매칭 점수가 가장 큰 카드뽑기

In [8]:
def map_category_list_to_key(category_list):
  keys = set()
  for category in category_list:
    for key, values in category_dict.items():
        if category in values:
            keys.add(key)
  return list(keys)

In [9]:
card

,Card Name,Annual_fee,Base Record,Category_List,category
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,[항공마일리지],[항공]
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"[쇼핑, 레저, 카페/베이커리, 편의점, 대형마트, 의료, 뷰티, 대중교통, 외식]","[뷰티, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화]"
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"[쇼핑, 주유, 카페/베이커리, 편의점, 대중교통, 간편결제, 대형마트, 외식, 통...","[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 항공..."
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"[대중교통, 간편결제, 편의점, 통신, 카페/베이커리, 대형마트, 주유, 외식, 쇼...","[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화..."
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"[포인트/캐시백, 카페/베이커리, 쇼핑, 통신, 대중교통, 영화, 편의점]","[쇼핑/간편결제, 납부, 문화, 교통/주유]"
...,...,...,...,...,...
161,LOCA PLATINUM(할인형),"국내 10만원, 해외 10만원, 가족겸용 없음",조건없음,"[카페/베이커리, 쇼핑, 뷰티, 대형마트, 편의점, 주유, 외식, 영화, 통신, 교...","[뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아]"
162,I-MILEAGE(아시아나),"국내 4만1천원, 해외 4만2천원",직전 1개월 합계 50만원 이상,"[항공마일리지, 카페/베이커리]",[항공]
163,The CJ-현대카드M Edition2,"국내 1만5천원, 해외 2만원",조건없음,"[쇼핑, 영화, 외식, 카페/베이커리, 포인트/캐시백]","[쇼핑/간편결제, 카페/베이커리, 문화]"
164,KB국민 그린카드(서울형),"국내전용 없음, 해외겸용 없음",직전 1개월 합계 20만원 이상,"[포인트/캐시백, 대중교통, 쇼핑, 대형마트]","[쇼핑/간편결제, 교통/주유]"


In [10]:
card[
    'category'
][3][1]

'뷰티'

In [11]:
interests.head()

,user_id,category,암묵적관심도,명시적관심도
0,1,교육/육아,1,0
1,1,문화,0,1
2,1,납부,0,1
3,1,항공,0,1
4,1,반려동물,1,0


In [49]:
card.head()

,Card Name,Annual_fee,Base Record,Category_List,category
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,[항공마일리지],[항공]
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"[쇼핑, 레저, 카페/베이커리, 편의점, 대형마트, 의료, 뷰티, 대중교통, 외식]","[뷰티, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화]"
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"[쇼핑, 주유, 카페/베이커리, 편의점, 대중교통, 간편결제, 대형마트, 외식, 통신, 교육, 의료, 레저, 육아, 문화, 반려동물, 뷰티, 영화, 항공마일리지]","[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 항공, 문화, 교육/육아]"
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"[대중교통, 간편결제, 편의점, 통신, 카페/베이커리, 대형마트, 주유, 외식, 쇼핑, 영화, 뷰티, 레저, 문화, 교육, 육아, 의료, 반려동물, 하이패스]","[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아]"
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"[포인트/캐시백, 카페/베이커리, 쇼핑, 통신, 대중교통, 영화, 편의점]","[쇼핑/간편결제, 납부, 문화, 교통/주유]"


In [54]:
for i, card_row in card.iterrows():
  print(i, card_row['category'])

0 ['항공']
1 ['뷰티', '카페/베이커리', '쇼핑/간편결제', '의료', '교통/주유', '문화']
2 ['반려동물', '뷰티', '납부', '카페/베이커리', '쇼핑/간편결제', '의료', '교통/주유', '항공', '문화', '교육/육아']
3 ['반려동물', '뷰티', '납부', '카페/베이커리', '쇼핑/간편결제', '의료', '교통/주유', '문화', '교육/육아']
4 ['쇼핑/간편결제', '납부', '문화', '교통/주유']
5 ['납부', '카페/베이커리', '쇼핑/간편결제', '의료', '교통/주유']
6 ['반려동물', '뷰티', '납부', '카페/베이커리', '쇼핑/간편결제', '의료', '교통/주유', '문화', '교육/육아']
7 ['교통/주유']
8 ['쇼핑/간편결제']
9 ['교통/주유', '쇼핑/간편결제', '문화']
10 ['쇼핑/간편결제', '교통/주유', '카페/베이커리', '문화']
11 ['납부', '카페/베이커리', '교통/주유', '문화', '교육/육아']
12 ['뷰티', '납부', '카페/베이커리', '쇼핑/간편결제', '의료', '교통/주유', '문화', '교육/육아']
13 ['납부', '카페/베이커리', '쇼핑/간편결제', '의료', '교통/주유', '문화', '교육/육아']
14 ['납부', '쇼핑/간편결제', '교통/주유', '문화', '교육/육아']
15 ['납부', '쇼핑/간편결제', '의료', '교통/주유', '문화', '교육/육아']
16 ['쇼핑/간편결제', '납부', '카페/베이커리']
17 ['쇼핑/간편결제', '납부', '카페/베이커리']
18 ['뷰티', '납부', '카페/베이커리', '쇼핑/간편결제', '의료', '교통/주유', '문화', '교육/육아']
19 ['쇼핑/간편결제']
20 ['쇼핑/간편결제', '납부', '카페/베이커리']
21 ['뷰티', '납부', '카페/베이커리', '쇼핑/간편결제', '의료', '교통/주유', '문화', '교육/육아']
22 ['뷰티', '납부

In [72]:

matching_card= []
for i, card_row in card.iterrows():
  total_score = 0
  for idx, user_row in interests.iterrows():
    score = 0
    # 카드의 혜택 카테고리와 사용자의 관심 카테고리 비교
    if user_row['category'] in card_row['category']:

      if user_row['암묵적관심도'] == 1:
        score += 0.7
      if user_row['명시적관심도'] == 1:
        score += 0.9

        # 매칭 데이터 기록
    matching_card.append({
            'user_id': user_row['user_id'],
            'category': user_row['category'],
            'Card_category': card_row['category'],
            'Card Name': card_row['Card Name'],
            'matching_score': score
        })

matching_card_df = pd.DataFrame(matching_card)

# 카드별 총 점수 계산
final_scores = matching_card_df.groupby(['user_id','Card Name']).agg({
    'matching_score': 'sum'
}).reset_index()

final_scores

,user_id,Card Name,matching_score
0,1,ALL 우리카드 Infinite,0.7
1,1,ALL 우리카드 Premium,1.6
2,1,BC 바로 K-패스 카드,2.5
3,1,BC 바로 MACAO 카드,0.7
4,1,BC 바로 리워드 플러스,3.2
...,...,...,...
1655,10,현대카드Z play,1.6
1656,10,현대카드Z work Edition2,2.3
1657,10,현대카드ZERO Edition3(포인트형),4.6
1658,10,현대카드ZERO Edition3(할인형),4.6


In [73]:
matching_card_df


,user_id,category,Card_category,Card Name,matching_score
0,1,교육/육아,[항공],하나 트래블로그 SKYPASS 신용카드,0.0
1,1,문화,[항공],하나 트래블로그 SKYPASS 신용카드,0.0
2,1,납부,[항공],하나 트래블로그 SKYPASS 신용카드,0.0
3,1,항공,[항공],하나 트래블로그 SKYPASS 신용카드,0.9
4,1,반려동물,[항공],하나 트래블로그 SKYPASS 신용카드,0.0
...,...,...,...,...,...
8129,10,반려동물,"[쇼핑/간편결제, 문화, 교통/주유]",KB국민 Green Wave 1.5℃ 카드,0.0
8130,10,항공,"[쇼핑/간편결제, 문화, 교통/주유]",KB국민 Green Wave 1.5℃ 카드,0.0
8131,10,교육/육아,"[쇼핑/간편결제, 문화, 교통/주유]",KB국민 Green Wave 1.5℃ 카드,0.0
8132,10,쇼핑/간편결제,"[쇼핑/간편결제, 문화, 교통/주유]",KB국민 Green Wave 1.5℃ 카드,0.7


In [74]:
final_scores[final_scores['user_id'] == 1].sort_values('matching_score')

,user_id,Card Name,matching_score
136,1,카드의정석 I&U+,0.0
86,1,삼성 iD ENERGY 카드,0.0
108,1,신한카드 MY CAR,0.0
0,1,ALL 우리카드 Infinite,0.7
41,1,LOCA MONEY (로카 머니),0.7
...,...,...,...
158,1,현대카드X,3.9
94,1,삼성 iD SIMPLE 카드,3.9
163,1,현대카드ZERO Edition3(포인트형),3.9
99,1,삼성카드 taptap S,3.9


In [75]:
card[card['Card Name']=='삼성카드 taptap S']['category']

,category
40,"[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아]"


In [15]:
interests[interests['user_id']==1]

,user_id,category,암묵적관심도,명시적관심도
0,1,교육/육아,1,0
1,1,문화,0,1
2,1,납부,0,1
3,1,항공,0,1
4,1,반려동물,1,0
5,1,쇼핑/간편결제,1,0


In [16]:
pd.set_option('display.max_colwidth', None)